In [ ]:
from transformers import BartForConditionalGeneration, TrainingArguments, Trainer , BartTokenizer, IntervalStrategy
import pandas as pd
import json
import torch


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Charger le tokenizer et le modèle
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large') # Charger le tokenizer BART
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large') # Charger le modèle BART

# Charger les données à partir du fichier JSON avec pandas
df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/data/train2.json') # Charger les données

# Préparation des données
def prepare_data(df):
    questions = df['question'].tolist() # Convertir les questions en liste
    answers = df['answer'].tolist() # Convertir les réponses en liste

    return questions, answers

questions, answers = prepare_data(df) # Préparer les données

# Convertir les textes en séquences
inputs = tokenizer(questions, truncation=True, padding='max_length', max_length=100, return_tensors='pt') # Tokeniser les questions
outputs = tokenizer(answers, truncation=True, padding='max_length', max_length=100, return_tensors='pt') # Tokeniser les réponses

# Préparer les données pour le modèle
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings # Encodages d'entrée
        self.labels = labels # Labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} # Obtenir l'élément d'index idx
        item['labels'] = torch.tensor(self.labels['input_ids'][idx]) # Obtenir les labels correspondants
        return item

    def __len__(self):
        return len(self.encodings.input_ids) # Retourner la longueur des encodages d'entrée

train_dataset = MyDataset(inputs, outputs) # Créer le jeu de données d'entraînement

# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir='./results',          # Répertoire de sortie
    num_train_epochs=1,              # Nombre total d'époques d'entraînement
    per_device_train_batch_size=2,   # Taille du lot par appareil pendant l'entraînement
    warmup_steps=50,                 # Nombre de pas de préchauffage pour le planificateur de taux d'apprentissage
    weight_decay=0.01,               # Force de la dégradation du poids
    logging_dir='./logs',            # Répertoire pour stocker les logs
    save_strategy=IntervalStrategy.EPOCH, # Stratégie de sauvegarde
)

# Créer le formateur et entraîner le modèle
trainer = Trainer(
    model=model,                         # Sélection du modèle à entraîner
    args=training_args,                  # Arguments d'entraînement
    train_dataset=train_dataset,         # Jeu de données d'entraînement
)

trainer.train() # Entraîner le modèle

In [ ]:
#Sauvegarder le modèle
trainer.save_model("/content/drive/MyDrive/Colab Notebooks/data/my_model3")

#Sauvegarder le tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/data/my_model3")